In [2]:
import pandas as pd
import numpy as np
import json
import re
import os
from datetime import datetime
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass, asdict, field

import dspy
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

/home/atothegodd/living_lab/living_lab/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# ============================================================
# 1. DATA LOADING & MODEL CONFIG
# ============================================================

In [3]:
# llm = dspy.LM(
#     model="ollama/gemma3:27b",
#     api_base=api_base,
#     temperature=0.0,
#     max_tokens=2000
# )

# llm = dspy.LM(
#     model="ollama/ministral-3:8b",
#     temperature=0.0,
#     max_tokens=2000
# )


In [4]:
students_df = pd.read_csv("students.csv")

# Configure LLM (Ollama or OpenAI)

LLM_URL = os.getenv("LLM_URL_2")
MODEL_NAME = os.getenv("MODEL_NAME")
API_KEY = 'sss'

llm = dspy.LM(
    model=f'ollama/{MODEL_NAME}',
    api_base=LLM_URL, 
    api_key=API_KEY, 
    cache=False
)


dspy.configure(lm=llm, tracๅk_usage=False)

dspy.settings.configure(lm=llm)

embedding_model = SentenceTransformer("all-MiniLM-L6-v2") # เดะหามาให้ครัช


RUN_ID = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
RUN_DIR = f"logs/run_{RUN_ID}"
os.makedirs(RUN_DIR, exist_ok=True)

MAX_PROJECTS = 5
print(f"✓ Setup complete. Run ID: {RUN_ID}")
print(f"✓ Students: {len(students_df)}")

✓ Setup complete. Run ID: 2026-01-29_18-43-27
✓ Students: 8


# ============================================================
# 2. CORE DATA STRUCTURES (STATE)
# ============================================================

In [5]:
@dataclass
class ProjectRole:
    """Represents a single human role in a project."""
    role: str
    quota: int
    responsibility: str
    required_skills: List[str]

    def validate(self) -> bool:
        return (
            len(self.role.strip()) > 0
            and self.quota > 0
            and len(self.responsibility.strip()) > 0
            and 3 <= len(self.required_skills) <= 6
        )

@dataclass
class ProjectSpecification:
    """Complete, machine-consumable project specification."""
    project_summary: str
    project_type: List[str]
    headcount: int
    duration_months: int
    roles: List[ProjectRole]
    assumptions: List[str]
    risks: List[str]

    def validate(self) -> Dict:
        errors = []
        total_quota = sum(r.quota for r in self.roles)
        if total_quota != self.headcount:
            errors.append(f"Quota sum ({total_quota}) != headcount ({self.headcount})")
        for role in self.roles:
            if not role.validate():
                errors.append(f"Invalid role: {role.role}")
        if not self.project_type or len(self.project_type) == 0:
            errors.append("project_type cannot be empty")
        return {"is_valid": len(errors) == 0, "errors": errors}

    @staticmethod
    def to_dict(spec: 'ProjectSpecification') -> Dict:
        """Convert ProjectSpecification to dictionary."""
        return {
            "project_summary": spec.project_summary,
            "project_type": spec.project_type,
            "headcount": spec.headcount,
            "duration_months": spec.duration_months,
            "roles": [
                {
                    "role": r.role,
                    "quota": r.quota,
                    "responsibility": r.responsibility,
                    "required_skills": r.required_skills
                }
                for r in spec.roles
            ],
            "assumptions": spec.assumptions,
            "risks": spec.risks
        }

@dataclass
class ConversationState:
    messages: List[Dict] = field(default_factory=list)
    current_intent: Optional[str] = None
    extracted_project: Optional[ProjectSpecification] = None


@dataclass
class AgentState:
    students_df: Optional[pd.DataFrame] = None 
    project_spec: Optional[ProjectSpecification] = None
    project_row: Optional[pd.Series] = None  # optional / legacy
    ranked_df: Optional[Dict[str, pd.DataFrame]] = None
    decision_df: Optional[Dict[str, pd.DataFrame]] = None
    capacity_ledger: Dict[str, int] = field(default_factory=dict)
    assigned_in_project: set = field(default_factory=set)
    upskill_df: Optional[pd.DataFrame] = None
    explanations: Optional[List[Dict]] = None
    validations: Optional[pd.DataFrame] = None
    conversation: ConversationState = field(default_factory=ConversationState)
    flags: Dict[str, bool] = field(default_factory=dict)
    steps: int = 0
    done: bool = False
    
    
@dataclass
class AgentStateView:
    ranked: bool
    decided: bool
    explained: bool
    validated: bool
    needs_upskill: bool

## State Abstraction & Action Space

In [6]:
def build_state_view(state: AgentState) -> AgentStateView:
    needs_upskill = False

    if isinstance(state.decision_df, dict):
        for df in state.decision_df.values():
            if df["decision"].isin(["UPSKILL", "REJECT"]).any():
                needs_upskill = True
                break

    return AgentStateView(
        ranked=state.ranked_df is not None,
        decided=state.decision_df is not None,
        explained=state.explanations is not None,
        validated=state.validations is not None,
        needs_upskill=needs_upskill,
    )


In [7]:
ALLOWED_ACTIONS = [
    "RANK",
    "DECIDE",
    "PERSIST",
]

In [8]:
def legal_actions(view: AgentStateView) -> List[str]:
    actions = []

    if not view.ranked:
        actions.append("RANK")
    elif not view.decided:
        actions.append("DECIDE")
    else:
        actions.append("PERSIST")

    return actions

# 3. Tools

In [9]:
class ToolRegistry:
    def __init__(self):
        self.tools = {}

    def register(self, name: str, tool):
        self.tools[name] = tool

    def get(self, name: str):
        return self.tools[name]


## Tool 1 : Ranking

In [10]:
ABBR_MAP = {
    "ml": "machine learning",
    "etl": "extract transform load",
    "cv": "computer vision",
    "nlp": "natural language processing",
    "dl": "deep learning",
}

def normalize_skill(text: str) -> str:
    text = text.lower().strip()
    tokens = re.split(r"[,\s]+", text)
    tokens = [ABBR_MAP.get(t, t) for t in tokens if t]
    return " ".join(tokens)


In [11]:
class SkillEmbedderTool:
    def __init__(self, model):
        self.model = model
        self.cache: Dict[str, np.ndarray] = {}

    def embed_text(self, text: str) -> np.ndarray:
        text = normalize_skill(text)
        if text not in self.cache:
            self.cache[text] = self.model.encode(
                text,
                normalize_embeddings=True
            )
        return self.cache[text]

    def embed_batch(self, texts: List[str]) -> List[np.ndarray]:
        result = []
        for t in texts:
            result.append(self.embed_text(t))
        return result
    
skill_embedder = SkillEmbedderTool(embedding_model)

In [ ]:
import pickle
from pathlib import Path

class SkillIndex:

    def __init__(
        self,
        skill_csv_path: str,
        embedder: SkillEmbedderTool,
        cache_path: str = "skill_index_cache.pkl",
    ):
        self.embedder = embedder
        self.cache_path = Path(cache_path)

        # ---------- LOAD FROM CACHE ----------
        if self.cache_path.exists():
            print("⚡ Loading cached SkillIndex ...")
            with open(self.cache_path, "rb") as f:
                data = pickle.load(f)

            self.df = data["df"]
            self.emb_matrix = data["emb_matrix"]

            print(f"✓ SkillIndex loaded ({len(self.df)} skills)")
            return

        # ---------- BUILD FROM SCRATCH (FIRST RUN ONLY) ----------
        print("⏳ Building SkillIndex (first run only) ...")

        self.df = pd.read_csv(skill_csv_path)

        assert "skill_id" in self.df.columns
        assert "skill_name" in self.df.columns

        # normalize
        self.df["skill_name_norm"] = self.df["skill_name"].apply(normalize_skill)

        # embed ONCE
        self.df["embedding"] = self.df["skill_name_norm"].apply(
            self.embedder.embed_text
        )

        self.emb_matrix = np.vstack(self.df["embedding"].values)

        # drop embedding column (ไม่ต้องเก็บซ้ำ)
        df_to_save = self.df.drop(columns=["embedding"])

        with open(self.cache_path, "wb") as f:
            pickle.dump(
                {
                    "df": df_to_save,
                    "emb_matrix": self.emb_matrix,
                },
                f
            )

        print(f"✓ SkillIndex cached ({len(self.df)} skills)")

    def search(
        self,
        raw_skill: str,
        top_k: int = 5,
        min_sim: float = 0.5,
    ) -> List[Dict]:
        """
        Map raw / noisy skill text to canonical skills
        """
        q_emb = self.embedder.embed_text(raw_skill)

        sims = self.emb_matrix @ q_emb
        idx = np.argsort(-sims)[:top_k]

        results = []
        for i in idx:
            if sims[i] < min_sim:
                continue
            results.append({
                "skill_id": self.df.iloc[i]["skill_id"],
                "skill_name": self.df.iloc[i]["skill_name"],
                "score": float(sims[i]),
            })

        return results


In [13]:
skill_index = SkillIndex(
    skill_csv_path="skill_node.csv",
    embedder=skill_embedder,
)

⚡ Loading cached SkillIndex ...
✓ SkillIndex loaded (9734 skills)


In [14]:
class StudentRankerTool:
    """
    Rank students per role using skill-level semantic matching + coverage
    - score: max cosine similarity (เดิม)
    - avg_cosine_similarity: average cosine similarity (เพิ่ม)
    """

    def rank_by_role(
        self,
        project_row: pd.Series,
        students_df: pd.DataFrame
    ) -> Dict[str, pd.DataFrame]:

        role_rankings = {}

        for role, required_skills in project_row["role_skill_map"].items():

            role_skill_embs = skill_embedder.embed_batch(required_skills)
            rows = []

            for _, student in students_df.iterrows():

                student_skills = [
                    normalize_skill(s)
                    for s in student["skills_text"].split(",")
                    if s.strip()
                ]

                student_skill_embs = skill_embedder.embed_batch(student_skills)

                # best similarity per required skill
                sims = [
                    max(np.dot(s_emb, r_emb) for s_emb in student_skill_embs)
                    for r_emb in role_skill_embs
                ]

                # ✅ metric เดิม (ยังใช้ตัดสินใจ)
                max_cos_sim = max(sims) if sims else 0.0

                # ➕ metric ใหม่ (holistic / KR1 / XAI)
                avg_cos_sim = float(np.mean(sims)) if sims else 0.0

                coverage = (
                    sum(s >= 0.6 for s in sims) / len(sims)
                    if sims else 0.0
                )

                rows.append({
                    "role": role,
                    "student_id": student["student_id"],
                    "student_name": student["name"],
                    "student_skills_text": student["skills_text"],

                    # เดิม
                    "score": round(max_cos_sim, 3),

                    # เพิ่ม
                    "avg_cosine_similarity": round(avg_cos_sim, 3),

                    "coverage": round(coverage, 2),
                    "current_assignments": student["current_assignments"],
                    "max_capacity": student["max_capacity"]
                })

            role_rankings[role] = (
                pd.DataFrame(rows)
                .sort_values("score", ascending=False)
                .reset_index(drop=True)
            )

        return role_rankings


In [15]:
student_ranker = StudentRankerTool()

## Tool 1.5 : +1 assigned สำหรับคนที่ accepted

In [16]:
## Tool X: StudentAssignmentUpdater
class StudentAssignmentUpdaterTool:
    """
    TOOL: Update student's current_assignments in database
    """

    def increment(self, student_id: str, delta: int = 1):
        """
        This should call real DB in production.
        For now: placeholder side-effect.
        """
        print(f"📌 [DB] Increment current_assignments for student {student_id} (+{delta})")

        # TODO: replace with real DB update
        # e.g. UPDATE students SET current_assignments = current_assignments + 1 WHERE student_id = ...
        
assignment_updater = StudentAssignmentUpdaterTool()

## Tool 2 : XAI rule-based + llm ตอน ถาม

In [17]:
class DecisionMakerTool:

    def __init__(self, max_projects: int = 5):
        self.max_projects = max_projects

    def apply_constraints(
        self,
        match_row: pd.Series,
        project_row: pd.Series
    ) -> Tuple[str, List[str]]:

        min_score = float(project_row["min_score"])

        if match_row["score"] < min_score:
            return "REJECT", ["Semantic match score below threshold"]

        if match_row["coverage"] < 0.1:
            return "UPSKILL", ["Insufficient skill coverage"]

        return "ACCEPT", ["Score and coverage meet project requirements"]



    def progressive_match(
        self,
        project_row: pd.Series,
        ranked_df: pd.DataFrame,
        capacity_ledger: Dict[str, int],
        assigned_in_project,
        assignment_updater,
    ) -> pd.DataFrame:

        quota = int(project_row["quota"])
        evaluated = []
        accepted_count = 0

        for _, row in ranked_df.iterrows():
            student_id = row["student_id"]
            current = capacity_ledger.get(student_id, 0)
            reason_codes = []

            # ---------- HARD CONSTRAINTS ----------
            if student_id in assigned_in_project:
                decision = "REJECT"
                reason_codes.append("ALREADY_ASSIGNED_IN_PROJECT")

            elif current >= self.max_projects:
                decision = "REJECT"
                reason_codes.append("SYSTEM_MAX_PROJECTS")

            elif current >= row["max_capacity"]:
                decision = "REJECT"
                reason_codes.append("PERSONAL_MAX_CAPACITY")

            # ---------- SOFT / SCORE ----------
            else:
                tmp_decision, _ = self.apply_constraints(row, project_row)

                if tmp_decision == "REJECT":
                    decision = "REJECT"
                    reason_codes.append("SCORE_BELOW_THRESHOLD")

                elif tmp_decision == "UPSKILL":
                    decision = "UPSKILL"
                    reason_codes.append("LOW_SKILL_COVERAGE")

                elif accepted_count < quota:
                    decision = "ACCEPT"
                else:
                    decision = "REJECT"
                    reason_codes.append("QUOTA_FILLED")

            # ---------- SIDE EFFECT ----------
            if decision == "ACCEPT":
                accepted_count += 1
                assigned_in_project.add(student_id)
                capacity_ledger[student_id] = current + 1
                assignment_updater.increment(student_id)

            evaluated.append({
                **row.to_dict(),
                "decision": decision,
                "reason_codes": reason_codes
            })

        return pd.DataFrame(evaluated)

In [18]:
decision_maker = DecisionMakerTool(max_projects=MAX_PROJECTS)

## Tool 3

In [19]:
class DecisionExplainerTool:
    def __init__(self, llm):
        self.llm = llm

    def explain_decision(self, context: Dict) -> str:
        prompt = f"""
คุณคือระบบ Explainable AI
หน้าที่ของคุณคือ "อธิบายค่าที่ระบบคำนวณไว้แล้ว" เท่านั้น

กฎที่ต้องทำตามอย่างเคร่งครัด:
1. อนุญาตให้กล่าวถึงเฉพาะข้อมูลที่อยู่ใน context เท่านั้น
2. ห้ามเพิ่ม skill ใหม่ ห้าม paraphrase ห้ามตีความแทนตัวเลข
3. matched_role_skills และ missing_role_skills ต้องอธิบายตามรายการที่ให้มาเท่านั้น
4. required_optional_skills ห้ามนำมาใช้เป็นเหตุผลในการปฏิเสธ
5. หากไม่มี system constraint ให้ระบุว่า "ไม่มีข้อจำกัดจากระบบ"

6. การอธิบาย metric ต้องทำตามกฎนี้อย่างเคร่งครัด:
   - หาก context มีค่า max_cosine_similarity
     ต้องแสดงค่าเชิงตัวเลข EXACT ตามที่ปรากฏใน context
   - หาก context มีค่า avg_cosine_similarity
     ต้องแสดงค่าเชิงตัวเลข EXACT ตามที่ปรากฏใน context
   - ❌ ห้ามใช้คำว่า "ไม่ระบุ", "โดยประมาณ", "ค่อนข้างสูง"
   - ❌ ห้ามอธิบายแทนตัวเลข
   - ❌ ห้ามหลีกเลี่ยงการแสดงค่า

7. ความหมายของ metric (ใช้เพื่ออธิบายเท่านั้น):
   - max_cosine_similarity:
     ระดับความใกล้เคียงสูงสุดของทักษะบางรายการ
   - avg_cosine_similarity:
     ค่าเฉลี่ยของความใกล้เคียงเชิงความหมายของทักษะทั้งหมดที่ตำแหน่งต้องการ
   ❌ ห้ามคำนวณเพิ่ม ❌ ห้ามเปรียบเทียบเชิงตัดสิน

ลำดับการอธิบาย (ต้องครบทุกหัวข้อ):
- ระบุตำแหน่ง (role)
- เหตุผลด้านทักษะ (ต้องกล่าวถึง max_cosine_similarity และ avg_cosine_similarity หากมี)
- เหตุผลด้านข้อจำกัดของระบบ
- สรุปผลการตัดสินใจ

Context:
{json.dumps(context, ensure_ascii=False, indent=2)}

ตอบเป็น bullet points ภาษาไทย
"""


        response = self.llm(
            messages=[{"role": "user", "content": prompt}],
            max_tokens=1000,
            temperature=0.0
        )

        # normalize output
        if response is None:
            return "ไม่สามารถสร้างคำอธิบายได้ในขณะนี้"

        if isinstance(response, list):
            first = response[0]
            if isinstance(first, str):
                return first.strip()
            if isinstance(first, dict):
                return first.get("content", "").strip()

        if isinstance(response, str):
            return response.strip()

        return "ไม่สามารถสร้างคำอธิบายได้ในขณะนี้"


In [20]:
explainer = DecisionExplainerTool(llm)

In [21]:
REASON_CODE_DEFS = {
    "PERSONAL_MAX_CAPACITY": (
        "ผู้สมัครมีจำนวนโปรเจกต์ที่กำลังรับผิดชอบอยู่ครบตาม "
        "max_capacity ที่ตั้งไว้ จึงไม่สามารถรับงานเพิ่มได้"
    ),
    "SYSTEM_MAX_PROJECTS": (
        "ผู้สมัครถึงจำนวนโปรเจกต์สูงสุดที่ระบบอนุญาต"
    ),
    "QUOTA_FILLED": (
        "ตำแหน่งนี้มีผู้ได้รับคัดเลือกครบตาม quota แล้ว"
    ),
    "SCORE_BELOW_THRESHOLD": (
        "คะแนน semantic ต่ำกว่าเกณฑ์ขั้นต่ำของตำแหน่งนี้"
    ),
    "LOW_SKILL_COVERAGE": (
        "ความครอบคลุมของทักษะต่ำกว่าเกณฑ์ที่ระบบกำหนด"
    )
}

In [22]:
def build_xai_context(row: pd.Series, project_row: pd.Series, role: str) -> Dict:

    # สมมติ: optional skill มีคำว่า "(optional)"
    raw_required = project_row["role_skill_map"][role]

    required_core_skills = []
    required_optional_skills = []

    for s in raw_required:
        if "optional" in s.lower():
            required_optional_skills.append(s.replace("(optional)", "").strip())
        else:
            required_core_skills.append(s)

    student_skills = [
        normalize_skill(s)
        for s in row["student_skills_text"].split(",")
        if s.strip()
    ]

    matched_skills = []
    missing_skills = []

    for r_skill in required_core_skills:
        r_emb = skill_embedder.embed_text(r_skill)

        best_match = None
        best_sim = 0.0

        for s_skill in student_skills:
            s_emb = skill_embedder.embed_text(s_skill)
            sim = float(np.dot(r_emb, s_emb))
            if sim > best_sim:
                best_sim = sim
                best_match = s_skill

        if best_sim >= 0.6:
            matched_skills.append({
                "required_skill": r_skill,
                "matched_student_skill": best_match,
                "similarity": round(best_sim, 2),
            })
        else:
            missing_skills.append(r_skill)

    return {
        "student_name": row["student_name"],
        "role": role,
        "decision": row["decision"],

        "score": round(row["score"], 3),
        "coverage": round(row["coverage"], 2),

        # 🔒 SKILL TRUTH ONLY
        "required_core_skills": required_core_skills,
        "required_optional_skills": required_optional_skills,
        "matched_role_skills": matched_skills,
        "missing_role_skills": missing_skills,

        "reason_codes": row["reason_codes"],
        "reason_code_definitions": {
            code: REASON_CODE_DEFS.get(code, "")
            for code in row["reason_codes"]
        }
    }


## Tool 4

In [23]:
class ValidateDecision(dspy.Signature):
    """
    Validate whether a decision is consistent WITHIN THE ROLE ONLY.

    Rules:
    - Judge only based on role_required_skills
    - matched_role_skills / missing_role_skills
    - Do NOT consider any other skills
    """

    context = dspy.InputField(desc="Role-scoped decision context")
    verdict = dspy.OutputField(desc="One of: OK, SUSPICIOUS")
    comment = dspy.OutputField(
        desc="Short reason (1-2 sentences), role-specific only"
    )

class DecisionValidatorTool(dspy.Module):

    def __init__(self):
        super().__init__()
        self.validate = dspy.Predict(ValidateDecision)

    def validate_decision(self, context: Dict) -> Tuple[str, str]:
        result = self.validate(context=str(context))
        return result.verdict, result.comment

    def validate_batch(
        self,
        decision_df_by_role: Dict[str, pd.DataFrame],
        project_row: pd.Series
    ) -> pd.DataFrame:

        validation_results = []

        for role, df in decision_df_by_role.items():
            for _, row in df.iterrows():
                if row["decision"] == "ACCEPT":
                    continue

                context = self._build_context(row, project_row, role)
                verdict, comment = self.validate_decision(context)

                validation_results.append({
                    "student_id": row["student_id"],
                    "role": role,
                    "decision": row["decision"],
                    "verdict": verdict,
                    "comment": comment
                })

        return pd.DataFrame(validation_results)

    @staticmethod
    def _build_context(match_row, project_row, role):
        xai_ctx = build_xai_context(match_row, project_row, role)

        return {
            "role": role,
            "decision": xai_ctx["decision"],
            "score": xai_ctx["score"],
            "coverage": xai_ctx["coverage"],
            "reason_codes": xai_ctx["reason_codes"],
            "role_required_skills": xai_ctx["role_required_skills"],
            "matched_role_skills": xai_ctx["matched_role_skills"],
            "missing_role_skills": xai_ctx["missing_role_skills"],
        }


validator = DecisionValidatorTool()

## Tool 5

In [24]:
class RecommendUpskill(dspy.Signature):
    """Recommend skill improvements based on student-project gap."""
    student_skills = dspy.InputField(desc="Current student skills")
    required_skills = dspy.InputField(desc="Project required skills")
    recommendation = dspy.OutputField(
        desc="Concrete skill gap analysis and learning recommendation (2-3 bullet points)"
    )

class UpskillCoachTool(dspy.Module):
    """
    TOOL 6: Generates personalized upskilling recommendations.
    For UPSKILL/REJECT decisions, proposes concrete learning path.
    """
    
    def __init__(self):
        super().__init__()
        self.recommend = dspy.ChainOfThought(RecommendUpskill)
    
    def generate_recommendation(self, student_skills: str, required_skills: str) -> str:
        result = self.recommend(
            student_skills=student_skills,
            required_skills=required_skills
        )
        return result.recommendation
    
    def coach_batch(self, decision_df: pd.DataFrame, project_row: pd.Series) -> pd.DataFrame:

        upskill_results = []

        for _, row in decision_df.iterrows():
            if row["decision"] not in ["UPSKILL", "REJECT"]:
                continue

            recommendation = self.generate_recommendation(
                student_skills=row["student_skills_text"],
                required_skills=project_row["required_skills_text"]
            )

            upskill_results.append({
                "student_id": row["student_id"],
                "student_name": row["student_name"],
                "decision": row["decision"],
                "recommendation": recommendation
            })

        return pd.DataFrame(upskill_results)

upskill_coach = UpskillCoachTool()

## Tool 6

In [25]:
class ResultsPersisterTool:
    """
    TOOL 7: Persists all matching results and artifacts.
    Outputs: decisions.csv, accepted.csv, explanations.txt, upskill_plans.csv, summary.json
    """
    
    def __init__(self, run_dir: str):
        self.run_dir = run_dir
        os.makedirs(run_dir, exist_ok=True)
    
    def persist_decisions(self, decision_df: pd.DataFrame) -> str:
        path = f"{self.run_dir}/decisions.csv"
        decision_df.to_csv(path, index=False)
        return path
    
    def persist_accepted(self, decision_df: pd.DataFrame, project_row: pd.Series, run_id: str) -> str:
        """Save only accepted assignments."""
        accepted = decision_df[decision_df["decision"] == "ACCEPT"].copy()
        accepted["project_id"] = project_row["project_id"]
        accepted["project_title"] = project_row["title"]
        accepted["run_id"] = run_id
        
        cols = ["run_id", "project_id", "project_title", "student_id", "student_name", "score"]
        path = f"{self.run_dir}/accepted.csv"
        accepted[cols].to_csv(path, index=False)
        return path
    
    def persist_explanations(self, explanations: List[Dict]) -> str:
        path = f"{self.run_dir}/explanations.txt"
        with open(path, "w", encoding="utf-8") as f:
            for e in explanations:
                explanation = e.get("explanation")
                if not explanation:
                    continue 
                f.write(f"[{e['decision']}] Student {e['student_id']}\n")
                f.write(explanation.strip() + "\n\n")
        return path

    
    def persist_validations(self, validation_df: pd.DataFrame) -> str:
        """Save validation results (SUSPICIOUS decisions only)."""
        suspicious = validation_df[validation_df["verdict"] == "SUSPICIOUS"]
        if suspicious.empty:
            return None
        
        path = f"{self.run_dir}/validation_alerts.txt"
        with open(path, "w", encoding="utf-8") as f:
            for _, row in suspicious.iterrows():
                f.write(
                    f"[SUSPICIOUS] Student {row['student_id']} "
                    f"(decision={row['decision']}): {row['comment']}\n"
                )
        return path
    
    def persist_upskill_plans(self, upskill_df: pd.DataFrame) -> str:
        """Save upskilling recommendations."""
        if upskill_df.empty:
            return None
        
        path = f"{self.run_dir}/upskill_plans.csv"
        upskill_df.to_csv(path, index=False)
        return path
    
    def persist_summary(self, project_row: pd.Series, decision_df: pd.DataFrame) -> str:
        # summarize per student (not per role)
        priority = {"ACCEPT": 3, "UPSKILL": 2, "REJECT": 1}

        per_student = (
            decision_df
            .assign(p=decision_df["decision"].map(priority))
            .sort_values("p", ascending=False)
            .groupby("student_id", as_index=False)
            .first()
        )

        summary = {
            "project_id": project_row["project_id"],
            "project_title": project_row["title"],
            "role_quotas": project_row["role_quota_map"],
            "accepted_count": int((per_student["decision"] == "ACCEPT").sum()),
            "rejected_count": int((per_student["decision"] == "REJECT").sum()),
            "upskill_count": int((per_student["decision"] == "UPSKILL").sum()),
        }

        path = f"{self.run_dir}/summary.json"
        with open(path, "w", encoding="utf-8") as f:
            json.dump(summary, f, indent=2, ensure_ascii=False)
        return path



persister = ResultsPersisterTool(RUN_DIR)

# add tool in toolregistry

In [26]:
tool_registry = ToolRegistry()
tool_registry.register("rank", student_ranker)
tool_registry.register("update_assignment", assignment_updater)
tool_registry.register("decide", decision_maker)
tool_registry.register("explain", explainer)
tool_registry.register("validate", validator)
tool_registry.register("upskill", upskill_coach)
tool_registry.register("persist", persister)

# Agent

## policy

In [27]:
class SelectNextAction(dspy.Signature):
    """
    Select the next agent action based on current abstract state.
    """
    state = dspy.InputField(desc="Current agent state (boolean flags)")
    allowed_actions = dspy.InputField(desc="List of allowed next actions")
    next_action = dspy.OutputField(
        desc="Choose exactly ONE action from allowed_actions"
    )


class ActionPolicy:
    def choose(self, state_view: AgentStateView, allowed: List[str]) -> str:
        return allowed[0]

## agent exec ทำตามคำสั่ง

In [28]:
class ActionExecutor:

    def __init__(self, tools: ToolRegistry):
        self.tools = tools
    
    @staticmethod
    def filter_by_capacity(students_df: pd.DataFrame) -> pd.DataFrame:
        """Hard filter: only students with available capacity."""
        return students_df[
            students_df["current_assignments"] < students_df["max_capacity"]
        ].reset_index(drop=True)
    
    
    def execute(self, action: str, state: AgentState) -> AgentState:
        action = action.upper()


        if action == "RANK":
            
            state.ranked_df = self.tools.get("rank").rank_by_role(
                state.project_row,
                state.students_df
            )

        elif action == "DECIDE":
            decisions = {}

            updater = self.tools.get("update_assignment")

            for role, ranked_df in state.ranked_df.items():
                quota = state.project_row["role_quota_map"][role]

                df = self.tools.get("decide").progressive_match(
                    project_row=pd.Series({
                        "quota": quota,
                        "min_score": state.project_row["min_score"]
                    }),
                    ranked_df=ranked_df,
                    capacity_ledger=state.capacity_ledger,
                    assigned_in_project=state.assigned_in_project,
                    assignment_updater=updater
                )

                decisions[role] = df

            state.decision_df = decisions



        elif action == "EXPLAIN":
            # EXPLAIN: Generate LLM explanations for non-accept decisions
            explanations = []
            for role, df in state.decision_df.items():
                for _, row in df.iterrows():
                    if row["decision"] != "ACCEPT":
                        ctx = build_xai_context(row, state.project_row, role)
                        explanations.append({
                            "role": role,
                            "student_id": row["student_id"],
                            "decision": row["decision"],
                            "explanation": self.tools.get("explain").explain_decision(ctx)
                        })
                state.explanations = explanations



        elif action == "VALIDATE":
            # VALIDATE: Check decision consistency
            state.validations = self.tools.get("validate").validate_batch(
                state.decision_df, 
                state.project_row
            )


        elif action == "UPSKILL":
            # UPSKILL: Generate learning recommendations for rejected/upskill students
            state.upskill_df = self.tools.get("upskill").coach_batch(
                state.decision_df,
                state.project_row
            )


        elif action == "PERSIST":
            def flatten_decisions(decision_df_by_role: Dict[str, pd.DataFrame]) -> pd.DataFrame:
                frames = []
                for role, df in decision_df_by_role.items():
                    tmp = df.copy()
                    tmp["role"] = role
                    frames.append(tmp)
                return pd.concat(frames, ignore_index=True)
            # PERSIST: Save all artifacts
            persister = self.tools.get("persist")
            flat_df = flatten_decisions(state.decision_df)
            
            persister.persist_decisions(flat_df)
            persister.persist_accepted(flat_df, state.project_row, RUN_ID)
            persister.persist_summary(state.project_row, flat_df) 

            # if state.explanations:
            #     persister.persist_explanations(state.explanations)

            if state.validations is not None and not state.validations.empty:
                persister.persist_validations(state.validations)

            if state.upskill_df is not None and not state.upskill_df.empty:
                persister.persist_upskill_plans(state.upskill_df)

        return state

### แปลง freetext

In [29]:
class ExtractProjectSpec(dspy.Signature):
    """
    Extract structured project specification from ambiguous description.
    MUST satisfy all validation constraints.
    """
    project_description = dspy.InputField(desc="Unstructured project description")
    headcount = dspy.InputField(desc="Total number of people needed")
    duration = dspy.InputField(desc="Project duration (e.g., '3 months')")

    specification = dspy.OutputField(
        desc=(
            "Return ONLY valid JSON with fields:\n"
            "- project_summary (string)\n"
            "- project_type (non-empty list of strings)\n"
            "- headcount (int, must equal input)\n"
            "- duration_months (int)\n"
            "- roles (list of objects, MUST NOT be empty)\n"
            "   Each role MUST have:\n"
            "     - role (non-empty string)\n"
            "     - quota (int >= 1)\n"
            "     - responsibility (non-empty string)\n"
            "     - required_skills (list of 3 to 6 strings)\n"
            "   Sum of all role.quota MUST equal headcount\n"
            "- assumptions (list of strings)\n"
            "- risks (list of strings)\n\n"
            "If constraints cannot be met, ADJUST roles so that quota sum equals headcount.\n"
            "Do not include explanations. JSON only."
        )
    )

class ProjectExtractorTool(dspy.Module):
    def __init__(self, max_retries: int = 2):
        super().__init__()
        self.extractor = dspy.Predict(ExtractProjectSpec)
        self.max_retries = max_retries

    def forward(self, project_description: str, headcount: int, duration: str) -> ProjectSpecification:
        last_error = None

        for attempt in range(self.max_retries + 1):
            result = self.extractor(
                project_description=project_description,
                headcount=headcount,
                duration=duration
            )

            try:
                spec_json = self._parse_json(result.specification)
                spec = self._build_specification(spec_json, headcount)
                return spec
            except Exception as e:
                last_error = e

        raise ValueError(f"Project extraction failed after retries: {last_error}")

    
    @staticmethod
    def _parse_json(json_str: str) -> Dict:
        """Extract JSON object from LLM response."""
        cleaned = re.sub(r"```(?:json)?", "", json_str)
        cleaned = cleaned.strip()
        start = cleaned.find("{")
        if start == -1:
            raise ValueError("No JSON object found")
        stack = []
        end = None
        for i in range(start, len(cleaned)):
            if cleaned[i] == "{":
                stack.append("{")
            elif cleaned[i] == "}":
                stack.pop()
                if not stack:
                    end = i + 1
                    break
        if end is None:
            raise ValueError("Unbalanced JSON braces")
        json_candidate = cleaned[start:end]
        try:
            return json.loads(json_candidate)
        except json.JSONDecodeError as e:
            raise ValueError(f"Failed to parse JSON: {e}")
    
    @staticmethod
    def _build_specification(spec_dict: Dict, headcount: int) -> ProjectSpecification:
        roles = []

        for role_dict in spec_dict.get("roles", []):
            required_skills = role_dict.get("required_skills", [])
            if isinstance(required_skills, str):
                required_skills = [s.strip() for s in required_skills.split(",") if s.strip()]

            role = ProjectRole(
                role=str(role_dict.get("role", "")).strip().upper(),
                quota=int(role_dict.get("quota", 0)),
                responsibility=str(role_dict.get("responsibility", "")).strip(),
                required_skills=required_skills
            )
            roles.append(role)

        spec = ProjectSpecification(
            project_summary=str(spec_dict.get("project_summary", "")).strip(),
            project_type=spec_dict.get("project_type", []),
            headcount=headcount,
            duration_months=int(spec_dict.get("duration_months", 0)),
            roles=roles,
            assumptions=spec_dict.get("assumptions", []),
            risks=spec_dict.get("risks", [])
        )

        validation = spec.validate()
        if not validation["is_valid"]:
            raise ValueError(f"Specification validation failed: {validation['errors']}")

        return spec

#### ============================================================
### 8. AUTONOMOUS AGENT (DETERMINISTIC ORCHESTRATION)
#### The agent is the main loop that:
#### 1. Receives a ProjectSpecification and list of students
#### 2. Delegates to tools via an executor
#### 3. Follows a rule-based planner for deterministic ordering
#### 4. Returns final matching decisions
#### ============================================================


In [30]:
def project_spec_to_row(spec: ProjectSpecification) -> pd.Series:
    role_skill_map = {}

    for r in spec.roles:
        canonical_skills = []

        for raw_skill in r.required_skills:
            hits = skill_index.search(raw_skill, top_k=3)

            # เก็บชื่อ canonical skill
            canonical_skills.extend(
                h["skill_name"].lower() for h in hits
            )

        # dedup
        role_skill_map[r.role.lower()] = list(set(canonical_skills))

    role_quota_map = {
        r.role.lower(): r.quota
        for r in spec.roles
    }

    return pd.Series({
        "project_id": "P_CHAT",
        "title": spec.project_summary[:50],
        "description": spec.project_summary,
        "role_skill_map": role_skill_map,     
        "role_quota_map": role_quota_map,
        "min_score": 0.6
    })


In [31]:
def enforce_legality(proposed: str, allowed: List[str]) -> str:
    if proposed in allowed:
        return proposed
    return allowed[0]

In [32]:
class AutonomousMatchingAgent:
    def __init__(self, policy, executor):
        self.executor = executor
        self.policy = policy

    def run(self, project_spec, students_df):
        project_row = project_spec_to_row(project_spec)

        state = AgentState(
            project_spec=project_spec,
            project_row=project_row,
            students_df=students_df,
            capacity_ledger=dict(
                zip(
                    students_df["student_id"],
                    students_df["current_assignments"]
                )   
            )
        )      

        max_steps = 10
        while not state.done and state.steps < max_steps:
            state.steps += 1

            view = build_state_view(state)
            allowed = legal_actions(view)
            proposed = self.policy.choose(view, allowed)
            action = enforce_legality(proposed, allowed)

            state = self.executor.execute(action, state)

            if action == "PERSIST":
                state.done = True

        return state


# interaction

In [33]:
class ClassifyIntent(dspy.Signature):
    user_message = dspy.InputField()
    intent = dspy.OutputField(
        desc="One of: CREATE_PROJECT, RUN_MATCHING, ASK_EXPLANATION, MODIFY_CONSTRAINT, CHAT"
    )

class IntentRouter(dspy.Module):
    def __init__(self):
        self.classifier = dspy.Predict(ClassifyIntent)
        
    def route(self, text: str, state: AgentState) -> str:
        text_l = text.lower()

        if state.flags.get("matched") and text_l.startswith("ทำไม"):
            return "ASK_EXPLANATION"


        if any(k in text_l for k in [
            "มีใครบ้าง",
            "รายชื่อทั้งหมด",
            "รายชื่อนักเรียน",
            "student ทั้งหมด",
            "นักเรียนทั้งหมด"
        ]):
            return "LIST_STUDENTS"


        if state.flags.get("project_ready") and any(k in text_l for k in ["จัดทีม", "assign", "match", "เริ่ม", "run"]):
            return "RUN_MATCHING"

        if any(k in text_l for k in ["โปรเจกต์", "project", "ทำระบบ", "อยากได้"]):
            return "CREATE_PROJECT"

        # fallback to LLM
        return self.classifier(user_message=text).intent


In [34]:
def extract_student_name(text: str) -> str:

    text = text.replace("ทำไม", "").replace("ไม่ผ่าน", "")
    text = text.replace("เพราะอะไร", "")
    return text.strip()

In [35]:
def find_student(decision_df_by_role, name):
    for role, df in decision_df_by_role.items():
        match = df[df["student_name"].str.contains(name, case=False)]
        if not match.empty:
            return match.iloc[0], role
    return None, None

In [36]:
def summarize_by_student(decision_df_by_role: Dict[str, pd.DataFrame]) -> pd.DataFrame:
    # flatten role -> rows
    flat = pd.concat(
        df.assign(role=role)
        for role, df in decision_df_by_role.items()
    )

    # priority: ACCEPT > UPSKILL > REJECT
    priority = {"ACCEPT": 3, "UPSKILL": 2, "REJECT": 1}

    final = (
        flat.assign(p=flat["decision"].map(priority))
            .sort_values("p", ascending=False)
            .groupby("student_id", as_index=False)
            .first()
    )

    return final


In [37]:
def chat_loop(agent: AutonomousMatchingAgent, extractor: ProjectExtractorTool):

    state = AgentState(students_df=students_df)
    router = IntentRouter()

    print("🤖 สวัสดีครับ ผมช่วยจัดทีมให้โปรเจกต์ได้ 😊")

    while True:
        user_text = input("👤 ")
        if user_text.lower() in ["exit", "quit"]:
            break

        intent = router.route(user_text, state)

        # --- Free chat (clarification loop) ---
        if intent == "CHAT":
            print("🤖 เล่าโปรเจกต์คร่าว ๆ ได้เลยครับ เดี๋ยวผมจัดการให้")

        # --- User describes project and we extract it ---
        elif intent == "CREATE_PROJECT":
            print("🤖 โอเคครับ ผมสรุปโปรเจกต์ให้นะ")
            try:
                spec = extractor(
                    project_description=user_text,
                    headcount=3,
                    duration="3 months"
                )
                
                state.conversation.extracted_project = spec
                state.flags["project_ready"] = True

                print("🤖 ผมเข้าใจว่าโปรเจกต์คือ:")
                print(json.dumps(ProjectSpecification.to_dict(spec), indent=2, ensure_ascii=False))

                print("🤖 ถ้าโอเค พิมพ์ว่า 'จัดทีมเลย' ได้ครับ")
            except Exception as e:
                print(f"🤖 ขอโทษนะครับ ผมสรุปไม่ได้ ({e})")

        elif intent == "RUN_MATCHING":
            if not state.conversation.extracted_project:
                print("🤖 ขอรายละเอียดโปรเจกต์ก่อนนะครับ")
                continue

            print("🤖 กำลังจัดทีมให้ครับ ⏳")

            final_state = agent.run(
                project_spec=state.conversation.extracted_project,
                students_df=students_df
            )

            state.flags["matched"] = True
            state.decision_df = final_state.decision_df
            state.project_row = final_state.project_row

            final_students = summarize_by_student(state.decision_df)

            accepted_df = final_students[final_students["decision"] == "ACCEPT"]
            rejected = (final_students["decision"] == "REJECT").sum()
            upskill  = (final_students["decision"] == "UPSKILL").sum()

            print("🤖 เสร็จแล้วครับ 🎉")
            print(f"✓ ยอมรับ: {len(accepted_df)}")
            print(f"✓ ปฏิเสธ: {rejected}")
            print(f"✓ อบรมเพิ่มเติม: {upskill}")

            if not accepted_df.empty:
                print("\n🤖 ทีมที่ได้รับคัดเลือก:")
                for _, row in accepted_df.iterrows():
                    print(f"- {row['student_name']} ({row['role']})")
            else:
                print("\n🤖 ยังไม่มีใครผ่านการคัดเลือก")

            print("\n🤖 ถ้าอยากรู้เหตุผลของใคร พิมพ์ว่า 'ทำไม <ชื่อ>' ได้เลยครับ")

            
        elif intent == "LIST_STUDENTS":
            names = state.students_df["name"].tolist()
            print("🤖 รายชื่อนักเรียนทั้งหมด:")
            for n in names:
                print("-", n)


        elif intent == "ASK_EXPLANATION":
            if not state.flags.get("matched"):
                print("🤖 ยังไม่ได้จัดทีมเลยครับ")
                continue

            name = extract_student_name(user_text)
            row, role = find_student(state.decision_df, name)

            if row is None:
                print("🤖 ผมหาคนนั้นไม่เจอครับ")
                continue

            ctx = build_xai_context(row, state.project_row, role)
            explanation = explainer.explain_decision(ctx)

            print(f"🤖 เหตุผลสำหรับ {row['student_name']}:")
            print(explanation)

        else:
            print("🤖 ผมยังไม่แน่ใจ ลองอธิบายโปรเจกต์เพิ่มอีกนิดได้ไหมครับ")

# Test

In [38]:
executor = ActionExecutor(tool_registry)
policy = ActionPolicy()

agent = AutonomousMatchingAgent(
    policy=policy,
    executor=executor
)

print("✓ Planner, Executor, and Agent initialized.")

✓ Planner, Executor, and Agent initialized.


In [39]:
extractor = ProjectExtractorTool()
chat_loop(agent, extractor)

🤖 สวัสดีครับ ผมช่วยจัดทีมให้โปรเจกต์ได้ 😊
🤖 เล่าโปรเจกต์คร่าว ๆ ได้เลยครับ เดี๋ยวผมจัดการให้


KeyboardInterrupt: Interrupted by user